In [1]:
import pandas as pd
import numpy as np

df = pd.read_excel('ks-projects-201612.xlsx')
df.head()

,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,duration
0,1000002330,The Songs of Adelaide & Abullah,Poetry,Publishing,GBP,2015-10-09 11:36:00,1000.0,2015-08-11 12:12:28,0.0,failed,0,GB,0.0,58.974676
1,1000004038,Where is Hank?,Narrative Film,Film & Video,USD,2013-02-26 00:20:50,45000.0,2013-01-12 00:20:50,220.0,failed,3,US,220.0,45.000000
2,1000007540,ToshiCapital Rekordz Needs Help to Complete Album,Music,Music,USD,2012-04-16 04:24:11,5000.0,2012-03-17 03:24:11,1.0,failed,1,US,1.0,30.041667
3,1000011046,Community Film Project: The Art of Neighborhoo...,Film & Video,Film & Video,USD,2015-08-29 01:00:00,19500.0,2015-07-04 08:35:03,1283.0,canceled,14,US,1283.0,55.683993
4,1000014025,Monarch Espresso Bar,Restaurants,Food,USD,2016-04-01 13:38:27,50000.0,2016-02-26 13:38:27,52375.0,successful,224,US,52375.0,35.000000


In [2]:
#keep only failed and successful
df = df.drop(df[df['state'] == 'canceled'].index)
df = df.drop(df[df['state'] == 'live'].index)
df = df.drop(df[df['state'] == 'undefined'].index)
df = df.drop(df[df['state'] == 'suspended'].index)

#projects longer than 60 days are no longer possible
df = df.drop(df[df['duration']>60].index)

#project ID number not important
df.drop('ID', axis=1, inplace=True)

#want to focus on playing cards
df = df.drop(df[df['category']!='Playing Cards'].index)

In [3]:
df.loc[df['state']=='failed','funded']=0
df.loc[df['state']=='successful','funded']=1

In [4]:
df.loc[df['country']!='US', 'USA']=0
df.loc[df['country']=='US', 'USA']=1

df.loc[df['currency']!='USD', 'currencyType']=0
df.loc[df['currency']=='USD', 'currencyType']=1

In [5]:
df.loc[df['pledged']<df['goal'], 'goalSuccess']=0
df.loc[df['pledged']==df['goal'], 'goalSuccess']=1
df.loc[df['pledged']>df['goal'], 'goalSuccess']=2

In [6]:
def duration(x):
    if x<=30:
        return 0
    if 30<x<=60:
        return 1
    
df['durationType'] = df['duration'].apply(duration)

In [7]:
def backers(x):
    if x <= 25:
        return 0
    if 25 < x <=50:
        return 1
    if x > 50:
        return 2

In [8]:
df['backersType'] = df['backers'].apply(backers)

In [9]:
df['avg_donation'] = df['pledged']/df['backers']
#can help determine when to reward people 

In [10]:
def avg_donation(x):
    if x < 50:
        return 0
    if 50 < x <=100:
        return 1
    if 100 < x <=500:
        return 2
    if 500 < x <=1000:
        return 3
    if 1000 < x <=5000:
        return 4
    if x > 5000:
        return 5

df['avg_donationType'] = df['avg_donation'].apply(avg_donation)

In [ ]:
#what else can create column for to check out correlation

In [11]:
df.head()

,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,...,usd pledged,duration,funded,USA,currencyType,goalSuccess,durationType,backersType,avg_donation,avg_donationType
143,"Crap Amidst Hilarity, vol. 1",Playing Cards,Games,USD,2016-03-22 00:36:45,7500.0,2016-01-22 01:36:45,942.25,failed,55,...,942.250000,59.958333,0.0,1.0,1.0,0.0,1,2,17.131818,0.0
199,Quick Query - Relaunch!,Playing Cards,Games,USD,2014-08-18 19:02:26,750.0,2014-07-19 19:02:26,1207.00,successful,56,...,1207.000000,30.000000,1.0,1.0,1.0,2.0,0,2,21.553571,0.0
204,Strain Wars: Playing cards,Playing Cards,Games,USD,2016-07-31 11:18:43,5000.0,2016-07-16 11:18:43,176.00,failed,6,...,100.000000,15.000000,0.0,1.0,1.0,0.0,0,0,29.333333,0.0
402,"Monster Mob, the fun card game for all the mon...",Playing Cards,Games,GBP,2016-07-21 22:29:45,5000.0,2016-06-21 22:29:45,80.00,failed,6,...,29.167912,30.000000,0.0,0.0,0.0,0.0,0,0,13.333333,0.0
739,Black Mint Playing Cards,Playing Cards,Games,USD,2016-03-02 17:42:32,10000.0,2016-02-01 17:42:32,125808.40,successful,2140,...,125808.400000,30.000000,1.0,1.0,1.0,2.0,0,2,58.788972,1.0


In [17]:
fewFunded=len(df[(df['backersType']==0) & (df['funded']==1)])
someFunded=len(df[(df['backersType']==1) & (df['funded']==1)])
manyFunded=len(df[(df['backersType']==2) & (df['funded']==1)])

fewBackers = len(df[df['backersType']==0])
someBackers = len(df[df['backersType']==1])
manyBackers = len(df[df['backersType']==2])

print(fewFunded/fewBackers)
print(someFunded/fewBackers)
print(manyFunded/fewBackers)

0.07615230460921844
0.09018036072144289
1.1102204408817635


In [12]:
pd.pivot_table(df, index = 'backersType', values='funded', aggfunc='mean')

,funded
backersType,
0,0.076152
1,0.312500
2,0.763085
